<a href="https://colab.research.google.com/github/swais23/bq-partition-gap-finder/blob/feature%2Fpartition-gap-finder/bq_partition_gaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Imports
import gspread
import ipywidgets as widgets
import pandas as pd
from google.auth import default
from google.cloud import bigquery
from google.colab import auth, syntax
from gspread.exceptions import SpreadsheetNotFound, WorksheetNotFound

auth.authenticate_user()
print("Authenticated")

In [ ]:
# @title Setup
project = "bigquery-public-data"  # @param ["bigquery-public-data"]
dataset = "wikipedia"  # @param ["wikipedia", "worldpop"]
execution_project = "my-bq-project"  # @param {"type":"string"}
partition_granularity = "daily"  # @param ["hourly","daily","monthly"]

if partition_granularity == "hourly":
    parse_function = "PARSE_TIMESTAMP"
    diff_function = "TIMESTAMP_DIFF"
    format_function = "FORMAT_TIMESTAMP"
    partition_format = "%Y%m%d%H"
    date_part = "HOUR"
    identifier = "hours"

else:
    parse_function = "PARSE_DATE"
    diff_function = "DATE_DIFF"
    format_function = "FORMAT_DATE"
    partition_format = "%Y%m%d" if partition_granularity == "daily" else "%Y%m"
    date_part = "DAY" if partition_granularity == "daily" else "MONTH"
    identifier = "days" if partition_granularity == "daily" else "months"

bq_client = bigquery.Client(project=execution_project)

sql = syntax.sql(
    f"""
    SELECT DISTINCT table_name
    FROM `{project}.{dataset}.INFORMATION_SCHEMA.PARTITIONS`
    WHERE SAFE.{parse_function}("{partition_format}", partition_id) IS NOT NULL
    ORDER BY 1
    """
)


def get_partitioned_tables(sql: str) -> widgets.SelectMultiple:

    tables = widgets.SelectMultiple(options=bq_client.query(sql).to_dataframe().table_name)

    return tables


# Enable table selection
tables = get_partitioned_tables(sql)
if tables.options:
    print("Select table(s):")
    display(tables)

else:
    print("No tables match the selected criteria.")

In [ ]:
# @title Find Partition Gaps
create_new_worksheet = True  # @param {"type":"boolean"}
output_worksheet = "Partition_Gaps"  # @param {"type":"string"}

creds, _ = default()
gc = gspread.authorize(creds)

selected_tables = tables.value
# Remove trailing comma from single-element tuple
if len(selected_tables) == 1:
    selected_tables = f"('{selected_tables[0]}')"

sql = syntax.sql(
    f"""
    WITH test AS (
        SELECT
            table_name,
            {parse_function}("{partition_format}", partition_id) AS partition_id,
            LEAD({parse_function}("{partition_format}", partition_id)) OVER (
                PARTITION BY table_name ORDER BY partition_id
            ) AS next_partition_id
        FROM `{project}.{dataset}.INFORMATION_SCHEMA.PARTITIONS`
        WHERE
            table_name IN {selected_tables}
            AND total_rows > 0
            AND partition_id NOT LIKE "%NULL%"
    )

    SELECT
        table_name,
        {format_function}("{partition_format}", partition_id) AS partition_id,
        {format_function}("{partition_format}", next_partition_id) AS next_partition_id,
        {diff_function}(next_partition_id, partition_id, {date_part}) - 1 AS {identifier}_missing
    FROM test
    WHERE {diff_function}(next_partition_id, partition_id, {date_part}) > 1
    ORDER BY table_name, partition_id
    """
)


def get_partition_gaps(sql: str) -> None:

    df = bq_client.query(sql).to_dataframe()

    if not df.empty:
        if create_new_worksheet:
            worksheet = gc.create(output_worksheet)
            sheet = worksheet.sheet1
            sheet.update_title(dataset)

        else:
            try:
                worksheet = gc.open(output_worksheet)
            except SpreadsheetNotFound:
                print(f"Error: Worksheet '{output_worksheet}' not found.")
                return

            try:
                sheet = worksheet.worksheet(dataset)
            except WorksheetNotFound:
                worksheet.add_worksheet(dataset, rows=1000, cols=26)
                sheet = worksheet.worksheet(dataset)

        sheet.update([df.columns.values.tolist()] + df.values.tolist())

        print(f"Some of the selected tables contain partition gaps: {sheet.url}")

    else:
        print("None of the selected tables contain partition gaps.")


get_partition_gaps(sql)